In [3]:
import Bio.PDB
parser = Bio.PDB.PDBParser(QUIET=True)
structure = parser.get_structure("structure", "1AKI.pdb")

In [20]:
atoms = list(structure.get_atoms())

In [22]:
coords = [atom.get_coord() for atom in atoms]

In [23]:
coords

[array([ 35.36500168,  22.34199905, -11.97999954]),
 array([ 35.89199829,  21.07299995, -11.42700005]),
 array([ 34.74100113,  20.26399994, -10.84399986]),
 array([ 33.94499969,  20.81299973, -10.08100033]),
 array([ 36.87200165,  21.43499947, -10.30599976]),
 array([37.45299911, 20.24799919, -9.56499958]),
 array([38.68799973, 20.64900017, -8.77499962]),
 array([39.05699921, 19.50799942, -7.83699989]),
 array([40.42300034, 19.77099991, -7.29899979]),
 array([ 34.73899841,  18.96100044, -11.04199982]),
 array([ 33.90299988,  17.99799919, -10.33300018]),
 array([34.79999924, 17.31200027, -9.29399967]),
 array([35.75899887, 16.60499954, -9.66499996]),
 array([ 33.13999939,  17.0340004 , -11.23200035]),
 array([ 32.25099945,  16.08399963, -10.43400002]),
 array([ 32.29399872,  17.7140007 , -12.28999996]),
 array([34.49100113, 17.54599953, -8.03800011]),
 array([35.18500137, 16.90299988, -6.91800022]),
 array([34.74200058, 15.44099998, -6.77099991]),
 array([33.52500153, 15.1619997 , -6.86